The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/FixedSizeArrays.ji for module FixedSizeArrays.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/PlotUtils.ji for module PlotUtils.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/PlotThemes.ji for module PlotThemes.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/Showoff.ji for module Showoff.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/StatsBase.ji for module StatsBase.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/NaNMath.ji for module NaNMath.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/FileIO.ji for module FileIO.


In [3]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/HDF5.ji for module HDF5.
INFO: Recompiling stale cache file /home/robert/.julia/lib/v0.5/JLD.ji for module JLD.


plot_outputs_Plots (generic function with 1 method)

In [14]:
#Basic parameters
## Basic parameters
maxiter=10^8;
max_time = 350;
max_epocs = 6;
printiters = true;
exacterror =true; 
tol = 10.0^(-3.0);
restol = 10.0^(-4.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [15]:
dataset = "w8a"
X,y  = loadDataset(dataset);
#Make this into a function, includes file in BigRidge.
# Return A = X'*X +lambda I.  b= X'*y
prob = Prob([],[],[],0,0,dataset);
sX = size(X);
prob.n=  sX[1];
lambda = 1/sX[2] ;
prob.A = X*X'+eye(prob.n)*lambda;

../data/w8a.jld


In [16]:
prob.b = X*y;
prob.xsol = prob.A\prob.b;
prob.lambda = lambda;

In [17]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [18]:
# Testing CG 
options.max_time = 2; options.restol = 10.0^(-8); 
method_name = "CG";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 52.63  |  0.26  | 0.0021 
 50  | 36.20  |  0.06  | 0.0041 
 75  | 27.95  |  0.03  | 0.0061 
100  | 16.05  |  0.02  | 0.0079 
125  | 12.31  |  0.02  | 0.0101 
150  | 8.11  |  0.01  | 0.0119 
175  | 5.45  |  0.01  | 0.0137 
200  | 2.18  |  0.00  | 0.0155 
225  | 1.32  |  0.00  | 0.0173 
250  | 0.85  |  0.00  | 0.0191 
275  | 0.46  |  0.00  | 0.0216 
300  | 0.18  |  0.00  | 0.0234 
325  | 0.10  |  0.00  | 0.0252 


In [19]:
# Testing grad descent
options.max_time = 2; options.restol = 10.0^(-8); options.maxiter=1000;
method_name = "grad";# gradient descent
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

grad
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 94.99  |  0.92  | 0.0087 
 50  | 92.02  |  1.60  | 0.0149 
 75  | 89.66  |  0.48  | 0.0210 
100  | 87.45  |  1.08  | 0.0282 
125  | 85.57  |  0.36  | 0.0343 
150  | 83.73  |  0.86  | 0.0409 
175  | 82.12  |  0.29  | 0.0470 
200  | 80.52  |  0.72  | 0.0531 
225  | 79.11  |  0.25  | 0.0592 
250  | 77.69  |  0.63  | 0.0656 
275  | 76.43  |  0.22  | 0.0716 
300  | 75.16  |  0.56  | 0.0777 
325  | 74.01  |  0.20  | 0.0837 
350  | 72.86  |  0.51  | 0.0899 
375  | 71.81  |  0.18  | 0.0960 
400  | 70.76  |  0.47  | 0.1020 
425  | 69.80  |  0.17  | 0.1083 
450  | 68.83  |  0.43  | 0.1155 
475  | 67.95  |  0.16  | 0.1212 
500  | 67.05  |  0.40  | 0.1273 
525  | 66.24  |  0.15  | 0.1333 
550  | 65.41  |  0.37  | 0.1394 
575  | 64.66  |  0.14  | 0.1457 
600  | 63.89  |  0.35  | 0.1517 
625  | 63.19  |  0.13  | 0.1578 
650  | 62.48  |  0.33  | 0.1638 
675  | 61.82  |  0.12  | 0.1739 
700  | 61.16  |  0.31  | 0.1

In [20]:
# Testing CD 
options.max_time = 2; options.restol = 10.0^(-8); options.maxiter=600;
method_name = "CD";# gradient descent
options.sketchsize = floor((prob.n)^(1/2));
output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 69.21  |  7.94  | 0.0159 
 50  | 52.38  |  6.15  | 0.0236 
 75  | 47.85  |  3.94  | 0.0339 
100  | 41.82  |  1.92  | 0.0490 
125  | 38.61  |  1.65  | 0.0659 
150  | 34.40  |  0.49  | 0.0720 
175  | 33.51  |  0.18  | 0.0780 
200  | 31.27  |  0.09  | 0.0841 
225  | 29.26  |  0.06  | 0.0899 
250  | 28.68  |  0.04  | 0.0957 
275  | 25.56  |  0.03  | 0.1015 
300  | 24.46  |  0.03  | 0.1095 
325  | 23.15  |  0.02  | 0.3093 
350  | 22.16  |  0.02  | 0.3373 
375  | 22.01  |  0.02  | 0.3513 
400  | 20.96  |  0.01  | 0.3573 
425  | 20.51  |  0.01  | 0.3597 
450  | 19.56  |  0.01  | 0.3621 
475  | 19.40  |  0.00  | 0.3644 
500  | 19.22  |  0.00  | 0.3710 
525  | 19.17  |  0.00  | 0.3733 
550  | 18.12  |  0.00  | 0.3758 
575  | 8.77  |  0.01  | 0.3782 
600  | 8.77  |  0.00  | 0.3814 


In [21]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [26]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

w8a


In [13]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];
# mask = [1,3,5]
# OUTPUTS[mask]